In [1]:
import megengine
import megengine.data as data
import megengine.data.transform as T
import megengine.module as M
import megengine.functional as F
import megengine.optimizer as optimizer
import megengine.jit as jit

In [3]:
class BasicBlock(M.Module):
    """每个ResNet18的Block都包含两层卷积"""
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        # 第一层卷积，接 BN 和 ReLU
        self.conv1 = M.ConvBnRelu2d(
            in_channels=in_channels, out_channels=out_channels,
            kernel_size=3, stride=stride, padding=1)
        # 第二层卷积，只接 BN
        self.conv2 = M.ConvBn2d(
            in_channels=out_channels, out_channels=out_channels,
            kernel_size=3, stride=1, padding=1)
        # 残差连接，当输入输出不一致/需要下采样时，用 ConvBn 实现变换
        if in_channels == out_channels and stride == 1:
            self.res_conn = M.Identity()
        else:
            self.res_conn = M.ConvBn2d(
                in_channels=in_channels, out_channels=out_channels,
                kernel_size=1, stride=stride)
    
    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = x + self.res_conn(identity)
        return F.relu(x)

In [4]:
class ResNet18(M.Module):
    def __init__(self):
        self.conv1 = M.ConvBnRelu2d(in_channels=3, out_channels=64,
                                    kernel_size=3, padding=1)
        # 8 个 BasicBlock，3 次下采样(stride=2)，共 8x2=16 层卷积
        self.blocks = M.Sequential(
            BasicBlock(64,  64),
            BasicBlock(64,  64),
            BasicBlock(64,  128, stride=2),
            BasicBlock(128, 128),
            BasicBlock(128, 256, stride=2),
            BasicBlock(256, 256),
            BasicBlock(256, 512, stride=2),
            BasicBlock(512, 512),
        )
        # 全连接分类器，输出维度为 10 类的预测
        self.classifier = M.Sequential(
            M.Dropout(0.2),
            M.Linear(512, 10)
        )
    
    def forward(self, x):
        # 1. 特征提取，输入为 Nx3x32x32 的图片，输出为 Nx512x4x4的张量(Tensor)
        x = self.conv1(x)
        x = self.blocks(x)
        # 2. 4x4平均池化(Average Pooling)
        x = F.avg_pool2d(x, 4)
        x = F.flatten(x, 1)
        # 3. 分类预测
        x = self.classifier(x)
        return x

In [22]:
def testing():
    # megengine内置CIFAR10的数据集(train=False)
    dataset = data.dataset.CIFAR10(root="/data", train=False)
    
    # 构造数据生产线
    dataloader = data.DataLoader(
        dataset,
        # Random换成Sequential, drop_last=False不漏掉任何一张图片
        sampler=data.SequentialSampler(dataset, batch_size=64, drop_last=False),
        transform=T.Compose([
            # T.RandomHorizontalFlip(),  测试的时候不进行数据增广
            T.Normalize(mean=0., std=255.),  # f(x) = (x - mean) / std
            T.ToMode("CHW"),
        ])
    )
    
    # 构造网络与输入
    model = ResNet18()
    image = megengine.tensor(dtype="float32")
    label = megengine.tensor(dtype="int32")
    
    state_dict = megengine.load("checkpoint.pkl")
    model.load_state_dict(state_dict)
      
    # 构造静态的计算图以充分发挥性能
    @jit.trace
    def test_func(image, label):
        # 前传
        loglikelihood = model(image)
        accuracy = F.accuracy(loglikelihood, label)
        return accuracy
        
    # 遍历一次测试数据集
    correct = 0
    for i, batch_data in enumerate(dataloader):
        image.set_value(batch_data[0])
        label.set_value(batch_data[1])

        acc1 = test_func(image, label)
        correct += acc1.item() * label.shape[0]
        
        if i % 50 == 0:
            print("step", i, "acc@1", acc1)
    print("Final accuracy =", correct / 10000 * 100, "%")

In [23]:
testing()

step 0 acc@1 Tensor([0.5938])
step 50 acc@1 Tensor([0.6719])
step 100 acc@1 Tensor([0.5938])
step 150 acc@1 Tensor([0.7188])
Final accuracy = 63.21 %
